In [21]:
# Настройка для исключения ошибки 
# SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async
# https://stackoverflow.com/questions/61926359/django-synchronousonlyoperation-you-cannot-call-this-from-an-async-context-u
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [22]:
# Загрузка необходимых функций и модели
from django.db.models import Count, Q
from django.db.models.functions import Lower
from main.models import VAK

In [23]:
# Получение публикаций, которые имеют дубликаты
# Приводим название к нижнему регистру и присваеваем этому полю lower_name
# Группруем по новому полю lower_name
# Считаем количетво публикаций и записываем количество в поле count
# Фильтруем записи, где count больше 1 (есть дубликаты)
# Получаем словарь с ключами lower_name и count
duplicate_names = VAK.objects.annotate(lower_name=Lower('Name')).values('lower_name').annotate(count=Count('lower_name')).filter(Q(count__gt=1))

# Формируем список из названием статей, которые хранятся в словаре duplicate_names с ключем 'lower_name'
list_duplicate = [dn['lower_name'] for dn in duplicate_names]

print(list_duplicate)

['sdf', 'test']


In [24]:
# Формируем queryset из статей из списка дубляжей, предварительно приводя все названия публикаций к нижнему регистру
vak_dupl = VAK.objects.annotate(lower_name=Lower('Name')).filter(lower_name__in=list_duplicate)
print(vak_dupl)

<QuerySet [<VAK: test>, <VAK: sdf>, <VAK: sdf>, <VAK: Test>]>
